In [4]:
import pandas as pd 
import numpy as np 
import tensorflow as tf 

import os 
import warnings
warnings.filterwarnings("ignore")

import gc 

In [5]:
from utils import *
from collections import Counter
from tqdm.autonotebook import tqdm

# 准备数据

In [11]:
class FieldHandler(object):
    def __init__(self, train_file_path, test_file_path=None, category_columns=[], continuation_columns=[]):
        '''
        :param train_file_path: 训练集文件文件名
        :param test_file_path: 测试集文件文件名
        :param category_columns: 类别型特征, list型
        :param continuation_columns: 连续型特征, list型
        '''
        self.train_file_path = None
        self.test_file_path = None
        self.feature_nums = 0   ## 不预留，不同field的nan值单独处理
        self.field_dict = {}


        self.category_columns = category_columns
        self.continuation_columns = continuation_columns

        if not isinstance(train_file_path, str):
            raise ValueError("train file path must str")
        if os.path.exists(train_file_path):
            self.train_file_path = train_file_path
        else:
            raise OSError("train file path isn't exists!")

        if test_file_path:
            if os.path.exists(test_file_path):
                self.test_file_path = test_file_path
            else:
                raise OSError("test file path isn't exists!")
        ## 读取数据
        self.read_data()

        ## 构建场到特征的字典
        self.build_field_dict()
        self.build_standard_scaler()
        ## 该模型只对类别型特征进行embedding
        self.field_nums = len(self.category_columns)
        self.num_feats = len(self.continuation_columns)

    def read_data(self):
        '''
        读取数据
        '''
        if self.train_file_path and self.test_file_path:
            train_df = pd.read_csv(self.train_file_path)[self.category_columns + self.continuation_columns]
            test_df = pd.read_csv(self.test_file_path)[self.category_columns + self.continuation_columns]
            self.df = pd.concat([train_df, test_df])
        else:
            self.df = pd.read_csv(self.train_file_path)[self.category_columns + self.continuation_columns]
            
        self.df[self.category_columns] = self.df[self.category_columns].astype(str)


    def build_field_dict(self):
        '''
        构建场到特征的映射关系
        '''
        for column in self.df.columns:
            if column in self.category_columns:
                ## 类别型特征中所有不同的值
                ## 不特殊对待缺失值
                cv = [f for f in self.df[column].unique()]
                ## 将每一种特征值对应到不同的特征标号，键表示对应的场
                self.field_dict[column] = dict(zip(cv, range(self.feature_nums, self.feature_nums+len(cv))))
                ## 对应特征数增加
                self.feature_nums += len(cv)

    def build_standard_scaler(self):
        '''
        对连续型特征进行标准化
        '''
        if self.continuation_columns:
            self.standard_scaler = StandardScaler()
            self.standard_scaler.fit(self.df[self.continuation_columns].values)
        else:
            self.standard_scaler = None

In [12]:
def transformation_data(file_path, field_hander, label=None):
    '''
    返回准备好的数据
    :param label: 目标值对应的列名
    :return:
    '''
    df_ = pd.read_csv(file_path)
    if label:
        if label in df_.columns:
            ## 获取对应的目标值
            labels = df_[[label]].values.astype("float32")
        else:
            raise KeyError(f"label '{label}' isn\'t exists!")
    df_v = df_[field_hander.category_columns]
    num_features = df_[field_hander.continuation_columns]
    
    del df_ 
    gc.collect()
    
    ## 对连续型特征和类别型特征的缺失值进行填充
    ## 对连续型特征进行归一化
    if field_hander.standard_scaler:
        num_features[field_hander.continuation_columns] = field_hander.standard_scaler.transform(num_features.values)
    ## 对连续型特征的缺失值进行填充
    num_features.fillna(0, inplace=True)

    ## 这个DataFrame用于记录每个类别型特征值和连续型特征对应的特征标号
    df_i = df_v.copy()

    for column in df_v.columns:
        print("cat: ", column)
        df_i[column] = df_i[column].map(field_hander.field_dict[column])
        ## 对于测试集，可能有的特征值没有在训练集中出现
        ## 第0号特征留给缺失值
        ##df_i[column].fillna(0, inplace=True)
        ## 对非缺失值赋值为1
        df_v[column] = 1
        ## 对值序列的缺失值用0填充
        ##df_v[column].fillna(0, inplace=True)

    cat_v = df_v.values.astype("float32")
    cat_i = df_i.values.astype("int32")
    num_features = num_features.values.astype("float32")
    features = {
        "cat_i": cat_i,
        "cat_v": cat_v,
        "num_feats": num_features
    }

    if label:
        return features, labels
    return features, None

In [13]:
data = pd.read_csv("../data/criteo/criteo_data.csv")

In [14]:
## 取出连续型特征和类别型特征
con = [f for f in data.columns if f.startswith("I")]
cat = [f for f in data.columns if f.startswith("C")]

In [15]:
# 定义fieldhandler对象
field_handler = FieldHandler(train_file_path="../data/criteo/criteo_data.csv", continuation_columns=con,
                           category_columns=cat)

In [16]:
# 获取要输入的特征和标签值
features, labels = transformation_data(file_path="../data/criteo/criteo_data.csv",
                                      field_hander=field_handler, label="Label")

cat:  C1
cat:  C2
cat:  C3
cat:  C4
cat:  C5
cat:  C6
cat:  C7
cat:  C8
cat:  C9
cat:  C10
cat:  C11
cat:  C12
cat:  C13
cat:  C14
cat:  C15
cat:  C16
cat:  C17
cat:  C18
cat:  C19
cat:  C20
cat:  C21
cat:  C22
cat:  C23
cat:  C24
cat:  C25
cat:  C26


# 定义基础配置

In [17]:
class Config(dict):
    def __init__(self, field_handler):
        self['seed'] = 2019 

        self['deep_layers'] = [32, 32]
        self['cross_layer_num'] = 3
        self['deep_layers_activation'] = "relu"
        self['dropout'] = [0.5, 0.5, 0.5]
        self['l2_reg'] = 0.05 
        self['lr'] = 0.01 
        self['max_to_keep'] = 5
        self['batch_size'] = 128 
        self['num_epochs'] = 5 
        self.threshold = 0.5 
        self.eval = 1000 
        self.checkpoint = 1000 
         
        
        self['cat_feat_size'] = field_handler.feature_nums
        self['embedding_size'] = 5
        self['field_size'] = field_handler.field_nums
        self['num_feat_size'] = field_handler.num_feats
        self['total_size'] = self['field_size']*self['embedding_size'] + self['num_feat_size']
        
        self['checkpoint_dir'] = "../model/DCN/checkpoint"
        self['summary_dir'] = "../model/DCN/summary"

# 构造模型

## 定义模型类

In [18]:
class DCN(BaseModel):
    def __init__(self, config):
        super().__init__(config)
        tf.set_random_seed(self.config["seed"])
        self.build_model()
        self.init_saver()
    
    def build_model(self):
        # 定义输入
        ## 类别型特征
        self.feat_index = tf.placeholder(tf.int32, shape=[None, None], name="feat_index")
        self.feat_value = tf.placeholder(tf.float32, shape=[None, None], name="feat_value")
        ## 数值型特征
        self.numeric_value = tf.placeholder(tf.float32, [None, None], name="num_value")
        ## 标签值
        self.labels = tf.placeholder(tf.float32, shape=[None, 1], name="label")
        ## dropout
        self.dropout_keep_deep = tf.placeholder(tf.float32, shape=[None], name="dropout_deep")
        self.is_training = tf.placeholder(tf.bool, name="is_training")


        self.weights = self._initialize_weights()

        ######################### Embedding ################################
        with tf.name_scope("Embedding"):
            ## 输出 [batch, field_size, embedding_size]
            self.embeddings = tf.nn.embedding_lookup(self.weights['feature_embedding'], self.feat_index)
            ## 形状 [batch, field_size, 1]
            feat_value = tf.expand_dims(self.feat_value, axis=2)
            self.embeddings = tf.multiply(self.embeddings, feat_value)

        # 将数值型特征和嵌入特征拼接 [batch, total_size]
        self.x0 = tf.concat([self.numeric_value, tf.reshape(self.embeddings, 
                                                            shape=[-1, self.config['field_size']*self.config['embedding_size']])],
                           axis=1)

        ######################## Deep NetWork ###############################

        self.deep_layers_activation = None
        if self.config['deep_layers_activation'] == "relu":
            self.deep_layers_activation = tf.nn.relu
        elif self.config['deep_layers_activation'] == "tanh":
            self.deep_layers_activation = tf.nn.tanh
        elif self.config['deep_layers_activation'] == "sigmoid":
            self.deep_layers_activation = tf.nn.sigmoid


        with tf.name_scope("DeepLayer"):
            self.y_deep = tf.nn.dropout(self.x0, self.dropout_keep_deep[0])

            ## 最终输出 [batch, deep_layers[-1]]
            for i in range(0, len(self.config["deep_layers"])):
                self.y_deep = tf.add(tf.matmul(self.y_deep, self.weights[f"deep_layer_{i}"]),
                                    self.weights[f"deep_bias_{i}"])
                self.y_deep = self.deep_layers_activation(self.y_deep)
                self.y_deep = tf.nn.dropout(self.y_deep, self.dropout_keep_deep[i+1])

        ######################### Cross NetWork ################################

        with tf.name_scope("CrossLayer"):
            ## 维度: [batch, total_size, 1]
            self._x0 = tf.expand_dims(self.x0, axis=2)
            x_l = self._x0
            for l in range(self.config['cross_layer_num']):
                '''
                x_l = tf.tensordot(tf.matmul(self._x0, x_l, transpose_b=True),
                                  self.weights[f"cross_layer_{l}"], 1) + self.weights[f"cross_bias_{l}"] + x_l
                这种计算方法会消耗比较大的空间，下面是优化方法
                '''
                ## 这里只会得到标量
                x_b = tf.tensordot(tf.transpose(x_l, [0, 2, 1]), self.weights[f"cross_layer_{l}"], 1)
                x_l = tf.multiply(self._x0, x_b) + self.weights[f"cross_bias_{l}"] + x_l
                
                
            self.cross_network_out = tf.squeeze(x_l, axis=2)

        ######################### Output Layer ##################################
        with tf.name_scope("Output"):
            concat_input = tf.concat([self.cross_network_out, self.y_deep], axis=1)
            self.logits = tf.add(tf.matmul(concat_input, self.weights['concat_projection']), 
                                self.weights['concat_bias'])

        ############################### Loss ###################################
        self.l2_loss = 0
        with tf.name_scope("Loss"):
            self.predictions = tf.nn.sigmoid(self.logits)
            losses = tf.losses.log_loss(self.labels, self.predictions)
            '''
            l2_regular = tf.contrib.layers.l2_regularizer(scale=self.config['l2_reg'])

            keys = [w for w in self.weights.keys() if "bias" not in w and "embedding" not in w]
            for key in keys:
                self.l2_loss += tf.contrib.layers.apply_regularization(l2_regular,
                                                                      [self.weights[key]])
            '''
            if self.config['l2_reg'] > 0: 
                self.l2_loss = tf.add_n([tf.nn.l2_loss(cand_var) for cand_var in tf.trainable_variables() 
                                    if "bia" and "embedding" not in cand_var.name])
            
            
            self.loss = tf.reduce_mean(losses) + self.config['l2_reg'] * self.l2_loss

        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            optimizer = tf.train.AdamOptimizer(self.config['lr'])
            self.train_op = optimizer.minimize(self.loss, global_step=self.global_step_tensor)
                
    
    # 初始化权重
    def _initialize_weights(self):
        weights = dict()
        ## embedding层权重
        weights["feature_embedding"] = tf.Variable(
            tf.random_normal([self.config['cat_feat_size'], self.config["embedding_size"]],
                            0.0, 0.01), name="cat_embeddings")
        weights["feature_bias"] = tf.Variable(
            tf.random_normal([self.config['cat_feat_size'], 1], 0.0, 1.0), name="cat_bias")
        
        ## 深层网络的权重
        num_layers = len(self.config['deep_layers'])  ## 表示网络的数量
        ### 第一层的权重特殊处理
        glorot = np.sqrt(2.0/(self.config['total_size']+self.config['deep_layers'][0]))
        weights['deep_layer_0'] = tf.Variable(
            tf.random_normal(shape=[self.config['total_size'], self.config['deep_layers'][0]],
                            mean=0.0, stddev=glorot, dtype=tf.float32), name="deep_layer_0")
        weights['deep_bias_0'] = tf.Variable(tf.constant(0.0, shape=[self.config['deep_layers'][0]]), 
                                             name="deep_bias_0")
        
        ### 其它层权重
        for i in range(1, num_layers):
            glorot = np.sqrt(2.0 / (self.config['deep_layers'][i-1]+self.config['deep_layers'][i]))
            weights[f"deep_layer_{i}"] = tf.Variable(
                tf.random_normal(shape=[self.config['deep_layers'][i-1], self.config['deep_layers'][i]],
                                mean=0.0, stddev=glorot, dtype=tf.float32), name=f"deep_layer_{i}")
            weights[f"deep_bias_{i}"] = tf.Variable(tf.constant(0.0, shape=[self.config['deep_layers'][i]]),
                                                   name=f"deep_bias_{i}")
        
        ## 交叉层
        for i in range(self.config["cross_layer_num"]):
            glorot = np.sqrt(2.0/(self.config['total_size']+1))
            weights[f"cross_layer_{i}"] = tf.Variable(
                tf.random_normal(shape=[self.config['total_size'], 1], mean=0, stddev=glorot,
                                dtype=tf.float32), name=f"cross_layer_{i}")
            weights[f"cross_bias_{i}"] = tf.Variable(tf.constant(0.0), name=f"cross_bias_{i}")
        
        
        ## 最后一层投影层
        input_size = self.config['total_size'] + self.config['deep_layers'][-1]
        glorot = np.sqrt(2.0/(input_size+1))
        weights['concat_projection'] = tf.Variable(
            tf.random_normal(shape=[input_size, 1], dtype=tf.float32), name="concat_projection")
        weights['concat_bias'] = tf.Variable(tf.constant(0.01), dtype=tf.float32)
        
        return weights
    
    
    def init_saver(self):
        self.saver = tf.train.Saver(max_to_keep=self.config["max_to_keep"])

## 定义训练类

In [19]:
class Trainer(BaseTrain):
    def __init__(self, sess, model, data, config, logger):
        super().__init__(sess, model, data, config, logger)
        self.train = data[0]
        self.eval = data[1]
        
    def train_epoch(self):
        # 定义迭代次数
        num_iter_per_epoch = self.train.length // self.config["batch_size"]
        
        for _ in tqdm(range(num_iter_per_epoch)):
            loss, metrics, step = self.train_step()
            train_acc, train_f_score = metrics["accuracy"], metrics["f_score"]
            
            ## 将训练过程的损失写入
            summaries_dict = {"loss": loss,
                             "acc": np.array(train_acc), 
                             "f_score": np.array(train_f_score)}
            self.logger.summarize(step, summarizer="train", scope="train_summary",
                                 summaries_dict=summaries_dict)
            
            if step % self.config.eval == 0: 
                print("Train - Step: {} | Loss: {} | Acc: {} | F1_Score: {}".format(
                    step, loss, train_acc, train_f_score))
                # 对测试测试集进行评估
                eval_losses = []
                eval_pred = []
                eval_true = []
                for batchEval in self.eval.iter_all(self.config["batch_size"]):
                    loss, predictions = self.eval_step(batchEval[0], batchEval[1], batchEval[2], batchEval[3])
                    eval_losses.append(loss)
                    eval_pred.extend(predictions)
                    eval_true.extend(batchEval[-1])
                getMetric = Metric(np.array(eval_pred), np.array(eval_true), self.config)
                metrics = getMetric.get_metrics()
                acc_mean = np.round(metrics["accuracy"], 5)
                gini_mean = np.round(metrics["gini_norm"], 5)
                loss_mean = np.round(np.mean(eval_losses), 5)
                print("Eval | Loss: {} | Accuracy: {} | Gini: {}".format(
                    loss_mean, acc_mean, gini_mean))
                summaries_dict = {"loss": np.array(loss_mean), 
                                 "accuracy": np.array(acc_mean), 
                                 "gini": np.array(gini_mean)}
                self.logger.summarize(step, summarizer="test", scope="test_summary",
                                     summaries_dict=summaries_dict)
            if step % self.config.checkpoint == 0: 
                self.model.save(self.sess)
        
    
    def train_step(self):
        batch_feat_i, batch_feat_v, batch_num_feats, batch_y = next(self.train.next_batch(self.config["batch_size"]))
        
        feed_dict = {self.model.feat_index: batch_feat_i, self.model.feat_value: batch_feat_v,
                    self.model.numeric_value: batch_num_feats, 
                    self.model.labels: batch_y,
                    self.model.dropout_keep_deep: self.config['dropout'],
                    self.model.is_training: True}
        
        _, loss, predictions, step = self.sess.run([self.model.train_op,
                                                   self.model.loss,
                                                   self.model.predictions, 
                                                   self.model.global_step_tensor],
                                                  feed_dict=feed_dict)
        getMetric = Metric(predictions, batch_y, self.config)
        metrics = getMetric.get_metrics()
        
        return loss, metrics, step
    
    def eval_step(self, *batch):
        feed_dict = {self.model.feat_index: batch[0], self.model.feat_value: batch[1],
                    self.model.numeric_value: batch[2],
                    self.model.labels: batch[3],
                    self.model.dropout_keep_deep: [1.0]*len(self.config['dropout']),
                    self.model.is_training: False}
        
        loss, predictions = self.sess.run([self.model.loss, self.model.predictions],
                                         feed_dict=feed_dict)
        return loss, predictions

## 数据包装类

In [20]:
class DataGenerator:
    def __init__(self, labels, *features):
        self.features = features
        self.labels = labels
        self.length = len(labels)
        ## 计算不同类别的比例
        unique = Counter(self.labels.ravel())
        self.ratio = [(key, value / self.length) for key, value in unique.items()]
        self.indices = []
        for key, _ in self.ratio:
            index = np.where(labels.ravel() == key)
            self.indices.append(index)
        
    def next_batch(self, batch_size):
        '''
        生成每一个batch的数据集
        '''
        choose = np.array([])
        for i in range(len(self.indices)):
            ## 按照在数据集中出现的比例采样
            idx = np.random.choice(self.indices[i][0],
                                   max(1, min(len(self.indices[i][0]), int(batch_size * self.ratio[i][1]))))
            '''
            ## 等比例采样
            idx = np.random.choice(self.indices[i][0],
                                  min(len(self.indices[i][0]), int(batch_size / len(self.indices))))
            '''
            choose = np.append(choose, idx)
        choose = np.random.permutation(choose).astype("int64")
        result = []
        for feat in self.features:
            result.append(feat[choose])
        result.append(labels[choose])
        yield result
        
    def iter_all(self, batch_size):
        '''
        按照batch迭代所有数据
        '''
        numBatches = self.length // batch_size + 1 
        for i in range(numBatches):
            result = []
            start = i*batch_size
            end = min(start+batch_size, self.length)
            for feat in self.features:
                result.append(np.asarray(feat[start:end]))
            result.append(np.asarray(self.labels[start:end]))
            yield result

In [21]:
# 划分数据集
train_idx = slice(0, int(len(labels)*0.8))
val_idx = slice(int(len(labels)*0.8), int(len(labels)))

train_cat_i, train_cat_v, train_num_feats, train_df_y = (features["cat_i"][train_idx], 
                                                         features["cat_v"][train_idx], 
                                                         features["num_feats"][train_idx],
                                                        labels[train_idx])

val_cat_i, val_cat_v, val_num_feats, val_df_y = (features["cat_i"][val_idx],
                                                features["cat_v"][val_idx],
                                                features["num_feats"][val_idx],
                                                labels[val_idx])

In [22]:
train = DataGenerator(train_df_y, train_cat_i, train_cat_v, train_num_feats)
val = DataGenerator(val_df_y, val_cat_i, val_cat_v, val_num_feats)

# 训练

In [23]:
def create_dirs(dirs):
    try:
        for dir_ in dirs: 
            if not os.path.exists(dir_):
                os.makedirs(dir_)
        return 0 
    except Exception as e:
        print("Creating directories error: {}".format(e))
        exit(-1)

In [24]:
def main():
    config = Config(field_handler)
    config["num_epochs"] = 2 
    create_dirs([config['summary_dir'], config['checkpoint_dir']])
    tf.reset_default_graph()
    session_conf = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
    session_conf.gpu_options.per_process_gpu_memory_fraction = 0.8 
    session_conf.gpu_options.allow_growth=True 
    
    model = DCN(config)
    sess = tf.Session(config=session_conf)
    pack_data = [train, val]
    logger = Logger(sess, config)
    trainer = Trainer(sess, model, pack_data, config, logger)
    trainer.train_all()

In [33]:
main()


当前正处于第1次迭代


Train - Step: 1000 | Loss: 0.6166831254959106 | Acc: 0.73228 | F1_Score: 0.37037
Eval | Loss: 0.5381799936294556 | Accuracy: 0.7614 | Gini: 0.46184
Saving model...
Model saved
Train - Step: 2000 | Loss: 0.5165849924087524 | Acc: 0.75591 | F1_Score: 0.43636
Eval | Loss: 0.5215799808502197 | Accuracy: 0.76585 | Gini: 0.48042
Saving model...
Model saved
Train - Step: 3000 | Loss: 0.5001413226127625 | Acc: 0.77953 | F1_Score: 0.41667
Eval | Loss: 0.5181900262832642 | Accuracy: 0.76751 | Gini: 0.48972
Saving model...
Model saved
Train - Step: 4000 | Loss: 0.42506060004234314 | Acc: 0.85039 | F1_Score: 0.64151
Eval | Loss: 0.5306100249290466 | Accuracy: 0.76836 | Gini: 0.4776
Saving model...
Model saved
Train - Step: 5000 | Loss: 0.4473000168800354 | Acc: 0.80315 | F1_Score: 0.46809
Eval | Loss: 0.5252400040626526 | Accuracy: 0.76806 | Gini: 0.47944
Saving model...
Model saved
Train - Step: 6000 | Loss: 0.4930572509765625 | Acc: 0.77953 | F1_Score: 0.41667
Eval | Loss: 0.5246300101280212 | A

Train - Step: 7000 | Loss: 0.4515524208545685 | Acc: 0.81102 | F1_Score: 0.53846
Eval | Loss: 0.5466200113296509 | Accuracy: 0.75428 | Gini: 0.46097
Saving model...
Model saved
Train - Step: 8000 | Loss: 0.45302814245224 | Acc: 0.84252 | F1_Score: 0.54545
Eval | Loss: 0.5668799877166748 | Accuracy: 0.75434 | Gini: 0.43185
Saving model...
Model saved
Train - Step: 9000 | Loss: 0.6132492423057556 | Acc: 0.81102 | F1_Score: 0.58621
Eval | Loss: 0.678380012512207 | Accuracy: 0.75132 | Gini: 0.42468
Saving model...
Model saved
Train - Step: 10000 | Loss: 0.5281270146369934 | Acc: 0.85827 | F1_Score: 0.64
Eval | Loss: 0.6008700132369995 | Accuracy: 0.75483 | Gini: 0.40901
Saving model...
Model saved
Train - Step: 11000 | Loss: 0.41974037885665894 | Acc: 0.80315 | F1_Score: 0.44444
Eval | Loss: 0.5525100231170654 | Accuracy: 0.76112 | Gini: 0.43948
Saving model...
Model saved
Train - Step: 12000 | Loss: 0.442089706659317 | Acc: 0.81102 | F1_Score: 0.57143
Eval | Loss: 0.5473600029945374 | Acc

In [25]:
## 不对nan值进行特殊处理，直接当做一个特征
main()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Use tf.cast instead.

当前正处于第1次迭代


Train - Step: 1000 | Loss: 0.5230885744094849 | Acc: 0.76378 | F1_Score: 0.25
Eval | Loss: 0.5120499730110168 | Accuracy: 0.77171 | Gini: 0.48846
Saving model...
Model saved
Train - Step: 2000 | Loss: 0.5670076012611389 | Acc: 0.77165 | F1_Score: 0.32558
Eval | Loss: 0.5231500267982483 | Accuracy: 0.76834 | Gini: 0.47225
Saving model...
Model saved
Train - Step: 3000 | Loss: 0.45411279797554016 | Acc: 0.79528 | F1_Score: 0.45833
Eval | Loss: 0.5318800210952759 | Accuracy: 0.76844 | Gini: 0.47352
Saving model...
Model saved
Train - Step: 4000 | Loss: 0.4850400686264038 | Acc: 0.80315 | F1_Score: 0.46809
Eval | Loss: 0.528219997882843 | Accuracy: 0.76277 | Gini: 0.46865
Saving model...
Model saved
Train - Step: 5000 | Loss: 0.5219179391860962 | Acc: 0.77953 | F1_Score: 0.44
Eval | Loss: 0.5261300206184387 | Accuracy: 0.76669 | Gini: 0.4809
Saving model...
Model saved
Train - Step: 6000 | Loss: 0.49063974618911743 | Acc: 0.79528 | F1_Score: 0.48
Eval | Loss: 0.5372800230979919 | Accuracy:

Train - Step: 7000 | Loss: 0.4394581913948059 | Acc: 0.80315 | F1_Score: 0.46809
Eval | Loss: 0.5674300193786621 | Accuracy: 0.7615 | Gini: 0.44235
Saving model...
Model saved
Train - Step: 8000 | Loss: 0.4340980052947998 | Acc: 0.82677 | F1_Score: 0.57692
Eval | Loss: 0.5304800271987915 | Accuracy: 0.76287 | Gini: 0.45965
Saving model...
Model saved
Train - Step: 9000 | Loss: 0.38260096311569214 | Acc: 0.86614 | F1_Score: 0.67925
Eval | Loss: 0.5470799803733826 | Accuracy: 0.76702 | Gini: 0.4629
Saving model...
Model saved
Train - Step: 10000 | Loss: 0.34689629077911377 | Acc: 0.85827 | F1_Score: 0.67857
Eval | Loss: 0.5615400075912476 | Accuracy: 0.75629 | Gini: 0.45107
Saving model...
Model saved
Train - Step: 11000 | Loss: 0.37602999806404114 | Acc: 0.85039 | F1_Score: 0.65455
Eval | Loss: 0.5833899974822998 | Accuracy: 0.75693 | Gini: 0.4091
Saving model...
Model saved
Train - Step: 12000 | Loss: 0.35748952627182007 | Acc: 0.83465 | F1_Score: 0.61818
Eval | Loss: 0.585940003395080